# 사용한 코드

In [ ]:
from PIL import Image
import os, glob
import numpy as np
import tensorflow as tf
from tensorflow import keras


def resize_images(img_path): #하위폴더 가위바위보 리사이즈로 변경
    images=glob.glob(img_path + "/scissor/*.jpg")  
    
    target_size=(28,28)
    for img in images:
        old_img=Image.open(img)
        new_img=old_img.resize(target_size,Image.ANTIALIAS)
        new_img.save(img, "JPEG")
    
    images=glob.glob(img_path + "/paper/*.jpg")  
    
    for img in images:
        old_img=Image.open(img)
        new_img=old_img.resize(target_size,Image.ANTIALIAS)
        new_img.save(img, "JPEG")
    
    images=glob.glob(img_path + "/rock/*.jpg")  
    
    for img in images:
        old_img=Image.open(img)
        new_img=old_img.resize(target_size,Image.ANTIALIAS)
        new_img.save(img, "JPEG")
    

def load_data(img_path, number_of_data=900):  # 가위바위보 이미지 개수 총합에 주의하세요.
    # 가위 : 0, 바위 : 1, 보 : 2
    img_size=28
    color=3
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)

    idx=0
    for file in glob.iglob(img_path+'/scissor/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1

    for file in glob.iglob(img_path+'/rock/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1  
    
    for file in glob.iglob(img_path+'/paper/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1
        
    print("학습데이터(x_train)의 이미지 개수는", idx,"입니다.")
    return imgs, labels

# 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper"
resize_images(image_dir_path) #학습용 데이터 리사이즈
image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/test"
resize_images(image_dir_path) #시험용 데이터 리사이즈

print("가위바위보 이미지 resize 완료!")

image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper" #원본 입력
(x_train, y_train)=load_data(image_dir_path)
x_train_norm = x_train/255.0   # 입력은 0~1 사이의 값으로 정규화
x_train_reshaped=x_train_norm.reshape( -1, 28, 28, 3)

imagetest_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/test" # 테스트값 입력
(x_test, y_test)=load_data(imagetest_dir_path)
x_test_norm = x_test/255.0  
x_test_reshaped=x_test_norm.reshape( -1, 28, 28, 3)

n_channel_1=16 #하이퍼파라메타
n_channel_2=64
n_dense=32
n_train_epoch=10

model=keras.models.Sequential() #학습네트웍 구축
model.add(keras.layers.Conv2D(n_channel_1, (3,3), activation='relu', input_shape=(28,28,3)))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(n_channel_2, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(n_dense, activation='relu'))
model.add(keras.layers.Dense(3, activation='softmax'))


model.summary()

model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy']) #컴파일

model.fit(x_train_reshaped, y_train, epochs=n_train_epoch)

test_loss, test_accuracy = model.evaluate(x_test_reshaped, y_test, verbose=2)#시험셋과 비교
print("test_loss: {} ".format(test_loss))
print("test_accuracy: {}".format(test_accuracy))



가위바위보 이미지 resize 완료!
학습데이터(x_train)의 이미지 개수는 625 입니다.
학습데이터(x_train)의 이미지 개수는 300 입니다.
Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
conv2d_48 (Conv2D)           (None, 26, 26, 16)        448       
_________________________________________________________________
max_pooling2d_48 (MaxPooling (None, 13, 13, 16)        0         
_________________________________________________________________
conv2d_49 (Conv2D)           (None, 11, 11, 64)        9280      
_________________________________________________________________
max_pooling2d_49 (MaxPooling (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_24 (Flatten)         (None, 1600)              0         
_________________________________________________________________
dense_48 (Dense)             (None, 32)                51232     
_________________________________________________________________
dense_49 (Dense)             (None, 3)                 99        
=================================================================
Total params: 61,059
Trainable params: 61,059
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
29/29 [==============================] - 0s 3ms/step - loss: 1.0669 - accuracy: 0.5777
Epoch 2/10
29/29 [==============================] - 0s 3ms/step - loss: 0.8061 - accuracy: 0.6351
Epoch 3/10
29/29 [==============================] - 0s 3ms/step - loss: 0.6904 - accuracy: 0.6458
Epoch 4/10
29/29 [==============================] - 0s 3ms/step - loss: 0.6078 - accuracy: 0.7231
Epoch 5/10
29/29 [==============================] - 0s 3ms/step - loss: 0.4950 - accuracy: 0.8272
Epoch 6/10
29/29 [==============================] - 0s 4ms/step - loss: 0.4124 - accuracy: 0.8916
Epoch 7/10
29/29 [==============================] - 0s 4ms/step - loss: 0.2971 - accuracy: 0.9385
Epoch 8/10
29/29 [==============================] - 0s 3ms/step - loss: 0.2515 - accuracy: 0.9313
Epoch 9/10
29/29 [==============================] - 0s 3ms/step - loss: 0.1337 - accuracy: 0.9760
Epoch 10/10
29/29 [==============================] - 0s 3ms/step - loss: 0.0902 - accuracy: 0.9903
29/29 - 0s - loss: 1.1156 - accuracy: 0.7778
test_loss: 1.1155623197555542 
test_accuracy: 0.7777777910232544

## 고찰

막연히 되겠거니 하고 한꺼번에 모아서 붙여서 작업하였더니 어디서 터진건지 찾을수가 없었다   
나눠서 작업하고 곳곳에 완료 메시지를 프린팅 해줌으로써 빠르고 안정적인 컴퓨팅이 가능할꺼 같다

## 회고

비록 인식률이 나쁘지만 주물러서 돌아가는 걸 보니 기쁩니다.   
또한 프로그램을 짜는 것도 짜는 거지만 원본데이터의 중요성이 엄청 크다는걸 느꼈습니다.